# Retrieval

The steps are:
1. Load documents
2. Split documents
3. Embed & store documents
4. Retrieve from documents

## 0. Prerequisites

In [1]:
!pip install langchain openai GitPython chromadb unstructured markdown tiktoken


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
search_keyword = "AlwaysBehavior"

## 1. Load documents

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

docs_loader = DirectoryLoader("pronto-docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
docs = docs_loader.load()

In [ ]:
docs

## 2. Split documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

docs_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=0)

In [ ]:
docs_splits = docs_splitter.split_documents(docs)

In [ ]:
len(docs_splits)

In [ ]:
from langchain_core.documents import Document
from langchain.text_splitter import TextSplitter

def get_splits_from_keyword(keyword: str, splitter: TextSplitter, documents: list[Document]):
    return list(filter(lambda x: keyword in x.page_content, splitter.split_documents(documents)))

In [ ]:
chunk_size = 100
chunk_overlap = 0

### 2.1 Recursive Character Text Splitter

In [ ]:
rct_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [ ]:
get_splits_from_keyword("ValueBehavior", rct_splitter, docs)

### 2.2 Recursive Character Text Splitter (from language Markdown)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

rctm_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [ ]:
get_splits_from_keyword("ValueBehavior", rctm_splitter, docs)

### 2.3 Recursive Character Text Splitter (custom separators)

In [ ]:
rctcs_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["#", "##", "###", "\n\n", "\n \n", "\n"]
)

In [ ]:
rctcs_splitter.split_documents(filter(lambda x: x.metadata["source"] == "pronto-docs/README.md", docs))

In [ ]:
splits = get_splits_from_keyword("ValueBehavior", rctcs_splitter, docs)
print(len(splits))
print(splits)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

ct_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator="\n"
)

In [ ]:
get_splits_from_keyword("connect", ct_splitter, docs)

## 3. Embed & store documents

This is where it gets interesting. We have different options for retrieving:
- Vectorstore
- Parent Document
- Multi-Vector
- Self Query
- Contextual Compression
- Time-Weighted Vectorstore
- Multi-Query Retriever
- Ensemble
- Long-Context Reorder

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores.chroma import Chroma

db = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="./chroma_db")

In [10]:
from langchain_core.retrievers import BaseRetriever

def check_for_keyword(keyword: str, retriever: BaseRetriever):
    rel_docs = retriever.get_relevant_documents(keyword)
    count_results = sum(map(lambda x: int(keyword in x.page_content), rel_docs))
    return str(count_results) + "/" + str(len(rel_docs))

### 3.1 Vectorstore

This is the basic method directly build into the vectorstore.

In [ ]:
vec_retriever = db.as_retriever()
vec_retriever_mmr = db.as_retriever(
    search_type="mmr",  # Also test "similarity",
)

In [ ]:
vec_retriever.get_relevant_documents(search_keyword)

In [ ]:
print(check_for_keyword(search_keyword, vec_retriever))
print(check_for_keyword(search_keyword, vec_retriever_mmr))

As you can see the vectorstore method delivers some correct result but performs poorly.

### 3.2 Parent Document Retriever

#### 3.2.1 Full Documents from smaller chunks

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

pdfd_vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

pdfd_store = InMemoryStore()
pdfd_retriever = ParentDocumentRetriever(
    vectorstore=pdfd_vectorstore,
    docstore=pdfd_store,
    child_splitter=docs_splitter,
)

In [ ]:
pdfd_retriever.add_documents(docs, ids=None)

In [ ]:
pdfd_retriever.get_relevant_documents(search_keyword)

In [ ]:
print(check_for_keyword(search_keyword, pdfd_retriever))

This method outright delivers a too large chunk size i.e. the whole document and is therefore not suitable.

#### 3.2.2 Larger chunks from smaller chunks

In [27]:
from langchain.retrievers.multi_vector import SearchType
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.vectorstores.chroma import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.embeddings import OpenAIEmbeddings

parent_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=400, chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=0)

pdlg_vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
pdlg_store = InMemoryStore()
pdlg_retriever = ParentDocumentRetriever(
    vectorstore=pdlg_vectorstore,
    docstore=pdlg_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

pdlg_retriever.add_documents(docs)

In [34]:
pdlg_retriever.get_relevant_documents("How to access values in a StoreBehaviour?")

[Document(page_content="The StateBehavior is the fundamental building block of a state machine. Each StateBehavior emits the signals  StateBehavior.entered()  and  StateBehavior.exited()  to communicate the state machine's state \n \n StoreBehavior \n Use the Godot meta properties to store state. You can configure it to store values in the global dictionary  G  and access it via  G.at(prop) . \n \n ValueBehavior", metadata={'source': 'pronto-docs/README.md'})]

In [32]:
from langchain.memory import ConversationSummaryMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI()
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=pdlg_retriever, memory=memory, verbose=True)
qa("How to access values in a StoreBehaviour?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: 
Follow Up Input: How to access values in a StoreBehaviour?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The StateBehavior is the fundamental building block of a state machine. Each StateBehavior emits the signals  StateBehavior.entered()  and  StateBehavior.exited()  to communicate the state machine's state 
 
 StoreBehavior 
 Use the Godot meta properties to store state. You can configure it to store values in the global dictionary  G  and access it via  G.at(prop) . 
 
 ValueB

{'question': 'How to access values in a StoreBehaviour?',
 'chat_history': [SystemMessage(content='')],
 'answer': 'Values in a StoreBehavior can be accessed using the global dictionary "G" and the "G.at(prop)" method. By configuring the StoreBehavior to store values in the global dictionary, the values can be accessed and manipulated throughout the game.'}

In [33]:
print(check_for_keyword(search_keyword, pdlg_retriever))

0/0


In [41]:
qa("How to delete the connection of two behaviors?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human asks how to access values in a StoreBehaviour. The AI explains that values in a StoreBehaviour can be accessed using the global dictionary "G" and the "G.at(prop)" method. By configuring the StoreBehaviour to store values in the global dictionary, the values can be accessed and manipulated throughout the game. The human then asks what behavior to use for finding a specific node in their scene. The AI suggests using the `get_node()` function, which allows the user to access nodes by their path in the scene hierarchy. The human then asks if there is a behavior for querying a specific node in their scene. The AI responds by explaining that the `get_node()` method can be used to query a specific node in the scene by specifying its path. An example code s

{'question': 'How to delete the connection of two behaviors?',
 'chat_history': [SystemMessage(content='The human asks how to access values in a StoreBehaviour. The AI explains that values in a StoreBehaviour can be accessed using the global dictionary "G" and the "G.at(prop)" method. By configuring the StoreBehaviour to store values in the global dictionary, the values can be accessed and manipulated throughout the game. The human then asks what behavior to use for finding a specific node in their scene. The AI suggests using the `get_node()` function, which allows the user to access nodes by their path in the scene hierarchy. The human then asks if there is a behavior for querying a specific node in their scene. The AI responds by explaining that the `get_node()` method can be used to query a specific node in the scene by specifying its path. An example code snippet is provided to demonstrate how to use `get_node()` to query a specific node. The human then asks what the FPS warn prop

In [46]:
pdlg_retriever.vectorstore.similarity_search("evaluate Bind Behavior")

[Document(page_content='Behavior \n Function \n \n BindBehavior', metadata={'doc_id': '9bcdb4bf-45e8-4cb5-8254-dd2536447bcb', 'source': 'pronto-docs/README.md'}),
 Document(page_content='Behavior \n Function \n \n BindBehavior', metadata={'doc_id': 'c32a35a9-cbe1-4f44-94cb-a6a2829e1de2', 'source': 'pronto-docs/README.md'}),
 Document(page_content='Behavior \n Function \n \n BindBehavior', metadata={'doc_id': 'fd742ccd-3901-43ea-8853-a226f89c504f', 'source': 'pronto-docs/README.md'}),
 Document(page_content='Behavior \n Function \n \n BindBehavior', metadata={'doc_id': '3baf6b9b-dacf-4ba0-96e9-8314086487cd', 'source': 'pronto-docs/README.md'})]